In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.prompt{display: none}
.prompt_container{display: none;}
.container{width: auto !important;}
.btn { font-size: 9px; }
#menubar { max-height: 2em; overflow: hidden;}
</style>
""")

# November 18

1. Do the Do Now!
2. Get on Slack!
3. Open up a new Jupyter notebook!
4. Get hyped!

In [15]:
html = """
<div class="article article--river old_article-river">
    <a href="official_6291315_133.html">
        <div class="article_media-container"></div>
        <h3 class="article_
        title">Joe Biden will not attend COP28,
        according to US official</h3>
    </a>
</div>
<div class="article article--river old
_article-river">
<a href="official_6291315_133.html">
        <div class="article_media-container"></div>
        <h3 class="article__title">
        Paul Lynch wins Booker Prize for his dystopian novel
        'Prophet Song'
        </h3>
    </a>
</div>
"""

In [16]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
soup

<html><body><div class="article article--river old_article-river">
<a href="official_6291315_133.html">
<div class="article_media-container"></div>
<h3 class="article_ title">Joe Biden will not attend COP28,
        according to US official</h3>
</a>
</div>
<div class="article article--river old _article-river">
<a href="official_6291315_133.html">
<div class="article_media-container"></div>
<h3 class="article__title">
        Paul Lynch wins Booker Prize for his dystopian novel
        'Prophet Song'
        </h3>
</a>
</div>
</body></html>

In [17]:
links = soup.find_all('a'
#     class_="article"
#    class_="article article---river old_article-river"
)

for link in links:
    print(link['href'])
#     print(link.get('href', 'missing'))

official_6291315_133.html
official_6291315_133.html


# Playwright!!!!!

In [18]:
import requests

url = "https://analytics.opensyllabus.org/record/works"

response = requests.get(url)

In [19]:
# class="sc-46f9336f-9 grYYaW"
response

<Response [200]>

In [20]:
# Try to grab all of the book titles from the page
from bs4 import BeautifulSoup

soup_doc = BeautifulSoup(response.content, 'html.parser')

In [21]:
soup_doc.find_all('p', class_='grYYaW')

[]

In [22]:
# p tag with a class of grYYaW
soup_doc.select('p.grYYaW')

# element with a class of grYYaW inside of td tag
soup_doc.select('.grYYaW')

[]

# Number One: did you get blocked?????

Some websites don't like Python scraping their sites.

Now I'm pretending to be a browser (but I'm still requests)

In [23]:
import requests

url = "https://analytics.opensyllabus.org/record/works"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

In [24]:
soup_doc.find_all('p', class_='grYYaW')

[]

## Number Two: is the content actually on the page????

We can see it ourselves.... but maybe it isn't making it through to requests/BeautifulSoup?

**Test:** Print out the HTML and... look at it! Or print out the text and... look at it!

In [25]:
soup_doc.text

'Open Syllabus AnalyticsOS HomeExploreAboutACCOUNTLearn moreHomeExploreSyllabiAccount RequiredTitlesAuthorsFieldsSchoolsCountriesPublishersLearning OutcomesAccount RequiredAboutAbout OS AnalyticsAboutPricingAccounts and SubscriptionsContact:analytics@opensyllabus.orgAbout Open SyllabusAbout OSTeamOther OS Sites and ServicesCourse MatcherCo-Assignment GalaxyPrint StoreBlogFind us on FacebookFind us on TwitterLearn MoreSIGNINAboutBlogShare SyllabiTerms and PoliciesPeopleContact'

In [26]:
print(soup_doc.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <title>
   Open Syllabus Analytics
  </title>
  <link href="/favicon.png" rel="icon" type="image/png"/>
  <link href="https://api.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.css" rel="stylesheet"/>
  <meta content="Map the college curriculum across 20.9 million syllabi" name="description"/>
  <meta content="https://analytics-beta.opensyllabus.org/" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="Open Syllabus Analytics" property="og:title"/>
  <meta content="Map the college curriculum across 20.9 million syllabi" property="og:description"/>
  <meta content="https://analytics-beta.opensyllabus.org/open-graph-analytics.png" property="og:image"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="opensyllabus.org" property="twitter:domain"/>
  <meta content="https://analytics-beta.opensyllabus.org/" property

It isn't there!!!! Maybe it's being loaded asynchronously or after page load.

- **JSON** is good
- **XML** is bad

XMLHttpRequest, this sounds awful. XHR = old API stuff

In [27]:
url = "https://api.opensyllabus.org/api/titles/?format=json&size=500"

response = requests.get(url)
data = response.json()

In [28]:
data.keys()

dict_keys(['syllabus_total', 'citation_total', 'work_total', 'works'])

In [29]:
len(data['works'])

500

In [30]:
data['works']

[{'id': '8590097889',
  'display_name': 'Calculus',
  'title': 'Calculus',
  'title_highlight': None,
  'subtitle': None,
  'subtitle_highlight': None,
  'authors': [{'id': 'James Stewart',
    'display_name': 'James Stewart',
    'author': 'James Stewart',
    'author_highlight': None}],
  'publisher': None,
  'year': 2012,
  'oer': False,
  'open_access': False,
  'description': 'This textbook provides a brief review of polynomials, trigonometric, exponential, and logarithmic functions, followed by discussion of limits, derivatives, and applications of differential calculus to real-world problem areas. This volume goes on the present an overview of integration, basic techniques for integration, a variety of applications of integration, and an introduction to (systems of) differential equations. In keeping with this emphasis on conceptual understanding, each exercise set in this three semester Calculus text begins with concept questions and each end-of-chapter review section that incl

In [31]:
# Take the list of dictionaries, turn it into a dataframe
import pandas as pd

df = pd.json_normalize(data['works'])
df.head()

id                                    display_name  \
0     8590097889                                        Calculus   
1  5042291807662                           The Elements of Style   
2  8065948691325  Human Anatomy and Physiology Laboratory Manual   
3  6828998132676                         Social Research Methods   
4  1958505133798                                         Biology   

                                            title title_highlight subtitle  \
0                                        Calculus            None     None   
1                           The Elements of Style            None     None   
2  Human Anatomy and Physiology Laboratory Manual            None     None   
3                         Social Research Methods            None     None   
4                                         Biology            None     None   

  subtitle_highlight                                            authors  \
0               None  [{'id': 'James Stewart', 'display_name': 'Jame...   
1               None  [{'id': 'William Strunk', 'display_name': 'Wil...   
2               None  [{'id': 'Elaine Nicpon Marieb', 'display_name'...   
3               None  [{'id': 'Alan Bryman', 'display_name': 'Alan B...   
4               None  [{'id': 'Neil Alexander Campbell', 'display_na...   

   publisher    year    oer  ...  \
0        NaN  2012.0  False  ...   
1        NaN  2017.0  False  ...   
2        NaN  2011.0  False  ...   
3        NaN  2001.0  False  ...   
4        NaN  2011.0  False  ...   

                                                urls publication_type rank  \
0  [https://openlibrary.org/books/OL25567233M/Cal...             book    2   
1  [https://ci.nii.ac.jp/ncid/BB11052040, https:/...             book    6   
2  [https://openlibrary.org/works/OL9585047W, htt...             book    3   
3  [https://openlibrary.org/works/OL1978437W, htt...             book    4   
4  [https://openlibrary.org/works/OL33792729W, ht...             book    8   

      score                                            sources citation_count  \
0  0.999635      [loc, openalex, ol, unlinked-citations, viaf]          15638   
1  0.998176  [goodbooks, loc, unlinked-citations, wiki, ope...          12660   
2  0.999270      [loc, openalex, ol, unlinked-citations, viaf]          12166   
3  0.998906      [loc, openalex, ol, unlinked-citations, viaf]          10795   
4  0.997446      [loc, openalex, ol, unlinked-citations, viaf]          10195   

              publisher.id   publisher.display_name      publisher.publisher  \
0                      NaN                      NaN                      NaN   
1                      NaN                      NaN                      NaN   
2                      NaN                      NaN                      NaN   
3  Oxford University Press  Oxford University Press  Oxford University Press   
4                      NaN                      NaN                      NaN   

   publisher.publisher_highlight  
0                            NaN  
1                            NaN  
2                            NaN  
3                            NaN  
4                            NaN  

[5 rows x 26 columns]

In [32]:
# Save it to Excel for non-data people
df.to_excel("books.xlsx", index=False)

In [33]:
df.head()

id                                    display_name  \
0     8590097889                                        Calculus   
1  5042291807662                           The Elements of Style   
2  8065948691325  Human Anatomy and Physiology Laboratory Manual   
3  6828998132676                         Social Research Methods   
4  1958505133798                                         Biology   

                                            title title_highlight subtitle  \
0                                        Calculus            None     None   
1                           The Elements of Style            None     None   
2  Human Anatomy and Physiology Laboratory Manual            None     None   
3                         Social Research Methods            None     None   
4                                         Biology            None     None   

  subtitle_highlight                                            authors  \
0               None  [{'id': 'James Stewart', 'display_name': 'Jame...   
1               None  [{'id': 'William Strunk', 'display_name': 'Wil...   
2               None  [{'id': 'Elaine Nicpon Marieb', 'display_name'...   
3               None  [{'id': 'Alan Bryman', 'display_name': 'Alan B...   
4               None  [{'id': 'Neil Alexander Campbell', 'display_na...   

   publisher    year    oer  ...  \
0        NaN  2012.0  False  ...   
1        NaN  2017.0  False  ...   
2        NaN  2011.0  False  ...   
3        NaN  2001.0  False  ...   
4        NaN  2011.0  False  ...   

                                                urls publication_type rank  \
0  [https://openlibrary.org/books/OL25567233M/Cal...             book    2   
1  [https://ci.nii.ac.jp/ncid/BB11052040, https:/...             book    6   
2  [https://openlibrary.org/works/OL9585047W, htt...             book    3   
3  [https://openlibrary.org/works/OL1978437W, htt...             book    4   
4  [https://openlibrary.org/works/OL33792729W, ht...             book    8   

      score                                            sources citation_count  \
0  0.999635      [loc, openalex, ol, unlinked-citations, viaf]          15638   
1  0.998176  [goodbooks, loc, unlinked-citations, wiki, ope...          12660   
2  0.999270      [loc, openalex, ol, unlinked-citations, viaf]          12166   
3  0.998906      [loc, openalex, ol, unlinked-citations, viaf]          10795   
4  0.997446      [loc, openalex, ol, unlinked-citations, viaf]          10195   

              publisher.id   publisher.display_name      publisher.publisher  \
0                      NaN                      NaN                      NaN   
1                      NaN                      NaN                      NaN   
2                      NaN                      NaN                      NaN   
3  Oxford University Press  Oxford University Press  Oxford University Press   
4                      NaN                      NaN                      NaN   

   publisher.publisher_highlight  
0                            NaN  
1                            NaN  
2                            NaN  
3                            NaN  
4                            NaN  

[5 rows x 26 columns]

In [34]:
df.to_csv("books.csv", index=False)

In [35]:
pd.read_csv("books.csv")#.to_csv("books.csv")

id                                    display_name  \
0       8590097889                                        Calculus   
1    5042291807662                           The Elements of Style   
2    8065948691325  Human Anatomy and Physiology Laboratory Manual   
3    6828998132676                         Social Research Methods   
4    1958505133798                                         Biology   
..             ...                                             ...   
495  8083128558443      Succeeding With Your Master's Dissertation   
496  1202591044442                 Writing Ethnographic Fieldnotes   
497  7224135181881                               Social Psychology   
498    94489353682               Psychology: Themes and Variations   
499  4612794966607      Accounting and Finance for Non-Specialists   

                                              title  title_highlight  \
0                                          Calculus              NaN   
1                             The Elements of Style              NaN   
2    Human Anatomy and Physiology Laboratory Manual              NaN   
3                           Social Research Methods              NaN   
4                                           Biology              NaN   
..                                              ...              ...   
495      Succeeding With Your Master's Dissertation              NaN   
496                 Writing Ethnographic Fieldnotes              NaN   
497                               Social Psychology              NaN   
498                                      Psychology              NaN   
499      Accounting and Finance for Non-Specialists              NaN   

                  subtitle  subtitle_highlight  \
0                      NaN                 NaN   
1                      NaN                 NaN   
2                      NaN                 NaN   
3                      NaN                 NaN   
4                      NaN                 NaN   
..                     ...                 ...   
495                    NaN                 NaN   
496                    NaN                 NaN   
497                    NaN                 NaN   
498  Themes and Variations                 NaN   
499                    NaN                 NaN   

                                               authors  publisher    year  \
0    [{'id': 'James Stewart', 'display_name': 'Jame...        NaN  2012.0   
1    [{'id': 'William Strunk', 'display_name': 'Wil...        NaN  2017.0   
2    [{'id': 'Elaine Nicpon Marieb', 'display_name'...        NaN  2011.0   
3    [{'id': 'Alan Bryman', 'display_name': 'Alan B...        NaN  2001.0   
4    [{'id': 'Neil Alexander Campbell', 'display_na...        NaN  2011.0   
..                                                 ...        ...     ...   
495  [{'id': 'John Biggam', 'display_name': 'John B...        NaN  2008.0   
496  [{'id': 'Robert M. Emerson', 'display_name': '...        NaN  2011.0   
497  [{'id': 'David G. Myers', 'display_name': 'Dav...        NaN  2013.0   
498  [{'id': 'Wayne Weiten', 'display_name': 'Wayne...        NaN  1989.0   
499  [{'id': 'Eddie McLaney', 'display_name': 'Eddi...        NaN  2004.0   

       oer  ...                                               urls  \
0    False  ...  ['https://openlibrary.org/books/OL25567233M/Ca...   
1    False  ...  ['https://ci.nii.ac.jp/ncid/BB11052040', 'http...   
2    False  ...  ['https://openlibrary.org/works/OL9585047W', '...   
3    False  ...  ['https://openlibrary.org/works/OL1978437W', '...   
4    False  ...  ['https://openlibrary.org/works/OL33792729W', ...   
..     ...  ...                                                ...   
495  False  ...  ['https://openlibrary.org/works/OL16970710W', ...   
496  False  ...  ['https://openlibrary.org/works/OL17919689W', ...   
497  False  ...  ['https://openlibrary.org/works/OL20827658W', ...   
498  False  ...  ['https://openlibrary.org/works/OL31450321W', ...   
499  False  ..

## Let's try another way: Browser automation with Playwright (and not Selenium!)

- **First step:** Installing playwright itself with `pip install playwright`
- **Second step:** Set up playwright to install the browsers, which is `!playwright install`

In [69]:
# print(soup_doc.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <title>
   Open Syllabus Analytics
  </title>
  <link href="/favicon.png" rel="icon" type="image/png"/>
  <link href="https://api.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.css" rel="stylesheet"/>
  <meta content="Map the college curriculum across 20.9 million syllabi" name="description"/>
  <meta content="https://analytics-beta.opensyllabus.org/" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="Open Syllabus Analytics" property="og:title"/>
  <meta content="Map the college curriculum across 20.9 million syllabi" property="og:description"/>
  <meta content="https://analytics-beta.opensyllabus.org/open-graph-analytics.png" property="og:image"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="opensyllabus.org" property="twitter:domain"/>
  <meta content="https://analytics-beta.opensyllabus.org/" property

In [36]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.3/33.3 MB 12.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [37]:
!playwright install

139.7 MiB [                    ] 0% 0.0s139.7 MiB [                    ] 0% 13.0s139.7 MiB [                    ] 0% 26.0s139.7 MiB [                    ] 0% 17.4s139.7 MiB [                    ] 0% 14.6s139.7 MiB [                    ] 0% 13.2s139.7 MiB [                    ] 1% 12.1s139.7 MiB [                    ] 1% 11.4s139.7 MiB [                    ] 1% 11.0s139.7 MiB [                    ] 1% 10.5s139.7 MiB [                    ] 1% 10.4s139.7 MiB [                    ] 2% 10.5s139.7 MiB [                    ] 2% 10.2s139.7 MiB [                    ] 2% 9.9s139.7 MiB [=                   ] 2% 9.6s139.7 MiB [=                   ] 2% 9.4s139.7 MiB [=                   ] 3% 9.3s139.7 MiB [=                   ] 3% 9.2s139.7 MiB [=                   ] 3% 11.4s139.7 MiB [=                   ] 3% 11.2s139.7 MiB [=                   ] 3% 11.0s139.7 MiB [=                   ] 4% 10.8s139.7 MiB [=                   ] 4% 10.7s139.7 MiB [=                   ] 4% 10.5s139.7 MiB [=          

139.7 MiB [=======             ] 34% 14.3s139.7 MiB [=======             ] 34% 14.2s139.7 MiB [=======             ] 35% 14.1s139.7 MiB [=======             ] 35% 14.0s139.7 MiB [=======             ] 35% 13.9s139.7 MiB [=======             ] 35% 14.0s139.7 MiB [=======             ] 35% 13.9s139.7 MiB [=======             ] 36% 13.9s139.7 MiB [=======             ] 36% 13.8s139.7 MiB [=======             ] 36% 13.7s139.7 MiB [=======             ] 36% 13.6s139.7 MiB [=======             ] 36% 13.5s139.7 MiB [=======             ] 37% 13.4s139.7 MiB [=======             ] 37% 13.3s139.7 MiB [========            ] 37% 13.2s139.7 MiB [========            ] 37% 13.1s139.7 MiB [========            ] 38% 13.0s139.7 MiB [========            ] 38% 12.9s139.7 MiB [========            ] 38% 12.8s139.7 MiB [========            ] 38% 12.7s139.7 MiB [========            ] 39% 12.6s139.7 MiB [========            ] 39% 12.5s139.7 MiB [========            ] 39% 12.4s139.7 MiB [========            ] 3

139.7 MiB [=================   ] 85% 2.5s139.7 MiB [=================   ] 86% 2.4s139.7 MiB [=================   ] 86% 2.3s139.7 MiB [=================   ] 87% 2.3s139.7 MiB [=================   ] 87% 2.2s139.7 MiB [==================  ] 87% 2.1s139.7 MiB [==================  ] 88% 2.1s139.7 MiB [==================  ] 88% 2.0s139.7 MiB [==================  ] 89% 1.9s139.7 MiB [==================  ] 89% 1.8s139.7 MiB [==================  ] 90% 1.7s139.7 MiB [==================  ] 90% 1.6s139.7 MiB [==================  ] 91% 1.5s139.7 MiB [==================  ] 91% 1.4s139.7 MiB [==================  ] 92% 1.4s139.7 MiB [==================  ] 92% 1.3s139.7 MiB [=================== ] 92% 1.3s139.7 MiB [=================== ] 92% 1.2s139.7 MiB [=================== ] 93% 1.2s139.7 MiB [=================== ] 93% 1.1s139.7 MiB [=================== ] 93% 1.0s139.7 MiB [=================== ] 94% 1.0s139.7 MiB [=================== ] 94% 0.9s139.7 MiB [=================== ] 94% 0.8s139.7 MiB [=====

80.6 MiB [                    ] 0% 0.0s80.6 MiB [                    ] 0% 7.3s80.6 MiB [                    ] 0% 6.3s80.6 MiB [                    ] 0% 5.6s80.6 MiB [                    ] 1% 5.0s80.6 MiB [                    ] 1% 5.1s80.6 MiB [                    ] 2% 5.0s80.6 MiB [=                   ] 2% 5.0s80.6 MiB [=                   ] 2% 5.1s80.6 MiB [=                   ] 3% 5.0s80.6 MiB [=                   ] 3% 4.9s80.6 MiB [=                   ] 4% 4.9s80.6 MiB [=                   ] 4% 4.8s80.6 MiB [=                   ] 5% 4.7s80.6 MiB [=                   ] 5% 4.8s80.6 MiB [=                   ] 5% 4.7s80.6 MiB [=                   ] 6% 4.7s80.6 MiB [=                   ] 6% 4.6s80.6 MiB [=                   ] 6% 4.7s80.6 MiB [=                   ] 7% 4.6s80.6 MiB [=                   ] 7% 5.7s80.6 MiB [==                  ] 7% 5.6s80.6 MiB [==                  ] 8% 5.5s80.6 MiB [==                  ] 8% 5.4s80.6 MiB [==                  ] 8% 5.3s80.6 MiB [==             

80.6 MiB [====================] 98% 0.1s80.6 MiB [====================] 99% 0.1s80.6 MiB [====================] 99% 0.0s80.6 MiB [====================] 100% 0.0s
Firefox 131.0 (playwright build v1465) downloaded to /Users/anushasubramanian/Library/Caches/ms-playwright/firefox-1465


68.4 MiB [                    ] 0% 0.0s68.4 MiB [                    ] 0% 5.4s68.4 MiB [                    ] 0% 5.0s68.4 MiB [                    ] 1% 4.6s68.4 MiB [                    ] 1% 5.0s68.4 MiB [                    ] 1% 4.9s68.4 MiB [                    ] 2% 4.9s68.4 MiB [                    ] 2% 4.7s68.4 MiB [=                   ] 2% 4.7s68.4 MiB [=                   ] 3% 4.6s68.4 MiB [=                   ] 3% 4.7s68.4 MiB [=                   ] 4% 4.6s68.4 MiB [=                   ] 4% 4.8s68.4 MiB [=                   ] 4% 4.7s68.4 MiB [=                   ] 5% 4.6s68.4 MiB [=                   ] 6% 4.5s68.4 MiB [=                   ] 6% 4.4s68.4 MiB [=                   ] 6% 4.3s68.4 MiB [=                   ] 7% 4.2s68.4 MiB [==                  ] 7% 4.2s68.4 MiB [==                  ] 8% 4.1s68.4 MiB [==                  ] 8% 5.0s68.4 MiB [==                  ] 9% 4.9s68.4 MiB [==                  ] 9% 5.0s68.4 MiB [==                  ] 9% 4.9s68.4 MiB [==             

In [38]:
print("Hello")

Hello


In [41]:
# BAD!!! Won't work in Jupyter
# from playwright.sync_api import sync_playwright

# GOOD!!! The only way to make it work in Jupyter
from playwright.async_api import async_playwright

In [42]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [43]:
# browser = playwright.chromium.launch(headless=False)
# browser.new_page()
# AttributeError: 'coroutine' object has no attribute
# you forgot to await something

In [44]:
page = await browser.new_page()

In [45]:
# This means you forgot something
page.goto("https://analytics.opensyllabus.org/record/works")

<coroutine object Page.goto at 0x10db8a020>

In [46]:
# You forgot await!!!
await page.goto("https://analytics.opensyllabus.org/record/works")

<Response url='https://analytics.opensyllabus.org/record/works' request=<Request url='https://analytics.opensyllabus.org/record/works' method='GET'>>

In [47]:
# What is the HTML after the JavaScript has run
# after you've talked to all the APIs
# and drawn all of the books in the list
html = await page.content()
html

'<!DOCTYPE html><html lang="en"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width"><title>Open Syllabus Analytics</title><link rel="icon" type="image/png" href="/favicon.png"><link href="https://api.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.css" rel="stylesheet"><meta name="description" content="Map the college curriculum across 20.9 million syllabi"><meta property="og:url" content="https://analytics-beta.opensyllabus.org/"><meta property="og:type" content="website"><meta property="og:title" content="Open Syllabus Analytics"><meta property="og:description" content="Map the college curriculum across 20.9 million syllabi"><meta property="og:image" content="https://analytics-beta.opensyllabus.org/open-graph-analytics.png"><meta name="twitter:card" content="summary_large_image"><meta property="twitter:domain" content="opensyllabus.org"><meta property="twitter:url" content="https://analytics-beta.opensyllabus.org/"><meta name="twitter:title" content="Open Syllabu

YOUR MISSION!!! Use this HTML to find the thing with the class of `grYYaW`

In [48]:
from bs4 import BeautifulSoup

# Take the HTML from currently fully loaded page
# notice: THERE'S NO REQUESTS ANYWHERE!!!!
soup_doc = BeautifulSoup(html)

In [49]:
# it wasnt working directly because the page of loading async and the html for this wasnt showing up 
# when we accessed it. 
soup_doc.find_all('p', class_='grYYaW')

[<p class="sc-46f9336f-9 grYYaW">Calculus</p>,
 <p class="sc-46f9336f-9 grYYaW">The Elements of Style</p>,
 <p class="sc-46f9336f-9 grYYaW">Human Anatomy and Physiology Laboratory Manual</p>,
 <p class="sc-46f9336f-9 grYYaW">Social Research Methods</p>,
 <p class="sc-46f9336f-9 grYYaW">Biology</p>,
 <p class="sc-46f9336f-9 grYYaW">Research Design</p>,
 <p class="sc-46f9336f-9 grYYaW">Doing Your Research Project: A Guide for First-Time Researchers in Education and Social Science</p>,
 <p class="sc-46f9336f-9 grYYaW">The Communist Manifesto</p>,
 <p class="sc-46f9336f-9 grYYaW">The Study Skills Handbook</p>,
 <p class="sc-46f9336f-9 grYYaW">Discovering Statistics Using IBM SPSS</p>,
 <p class="sc-46f9336f-9 grYYaW">A Manual for Writers of Term Papers, Theses, and Dissertations: Chicago Style for Students and Researchers</p>,
 <p class="sc-46f9336f-9 grYYaW">Orientalism</p>,
 <p class="sc-46f9336f-9 grYYaW">They Say/I Say</p>,
 <p class="sc-46f9336f-9 grYYaW">Imagined Communities: Reflect

In [50]:
titles = soup_doc.find_all('p', class_='grYYaW')
for title in titles:
    print(title.text.strip())

Calculus
The Elements of Style
Human Anatomy and Physiology Laboratory Manual
Social Research Methods
Biology
Research Design
Doing Your Research Project: A Guide for First-Time Researchers in Education and Social Science
The Communist Manifesto
The Study Skills Handbook
Discovering Statistics Using IBM SPSS
A Manual for Writers of Term Papers, Theses, and Dissertations: Chicago Style for Students and Researchers
Orientalism
They Say/I Say
Imagined Communities: Reflections on the Origin and Spread of Nationalism
English Grammar in Use
Introduction to Algorithms
Molecular Biology of the Cell: The Problems Book
A Pocket Style Manual
The Craft of Research
Frankenstein
Leviathan
Nicomachean Ethics
A Writer's Reference
Artificial Intelligence: A Modern Approach
Discipline and Punish: The Birth of the Prison
The Prince
The Art of Public Speaking
Give Me Liberty!: An American History
Heart of Darkness
Marketing Management
The Republic
Research Methods for Business Students
Critical Thinking S

In [51]:
#rows = soup_doc.find_all(class_='hZcpkR')
# Find things with the class of hZcpkR OR the class of sDMUI
rows = soup_doc.select('.hZcpkR, .sDMUI')

# Find table rows <tr> immediately inside of something
# with the class of iwTYWZ
rows = soup_doc.select('.iwTYWZ > tr')

data_all = []

# it's better to ask for forgiveness than permission
for row in rows:
    # print("-----")
    # print(row.text.strip())
    
    data = {}
    
    # sc-46f9336f-9 grYYaW
    # print out the title
    # select by class of grYYaW
    try:
        data['title'] = row.find(class_='grYYaW').text
    except:
        print("Could not pull title")
    
    # rank, authors, appearances, score
    try:
        data['rank'] = row.find(class_='hhrvZq').text
    except:
        print("Could not pull rank")
    
    # find the 3rd cell
    try:
        data['appearances'] = row.find_all('td')[2].text
    except:
        print("Could not pull appearances")
    
    # find the 4th cell
    try:
        data['score'] = row.find_all('td')[3].text
    except:
        print("Could not pull score")

    # print(data)
    data_all.append(data)

df = pd.json_normalize(data_all)
df.head()

title rank appearances score
0                                        Calculus    1      15,638   100
1                           The Elements of Style    2      12,660   100
2  Human Anatomy and Physiology Laboratory Manual    3      12,166   100
3                         Social Research Methods    4      10,795   100
4                                         Biology    5      10,195   100

In [52]:
from io import StringIO

tables = pd.read_html(StringIO(html))
tables[0]

RANK                                             TITLES  APPEARANCES  \
0      1                     CalculusJames Stewart10015,638        15638   
1      2       The Elements of StyleWilliam Strunk10012,660        12660   
2      3  Human Anatomy and Physiology Laboratory Manual...        12166   
3      4  Social Research MethodsAlan BrymanOxford Unive...        10795   
4      5            BiologyNeil Alexander Campbell10010,195        10195   
5      6        Research DesignJohn W. CreswellSAGE1009,615         9615   
6      7  Doing Your Research Project: A Guide for First...         9309   
7      8           The Communist ManifestoKarl Marx1009,273         9273   
8      9  The Study Skills HandbookStella CottrellPalgra...         8759   
9     10  Discovering Statistics Using IBM SPSSAndy P. F...         8724   
10    11  A Manual for Writers of Term Papers, Theses, a...         8443   
11    12                  OrientalismEdward W. Said1008,420         8420   
12    13  They Say/I SayCathy BirkensteinW. W. Norton100...         8246   
13    14  Imagined Communities: Reflections on the Origi...         8198   
14    15  English Grammar in UseRaymond MurphyCambridge ...         7903   
15    16     Introduction to AlgorithmsT. H. Corman1007,729         7729   
16    17  Molecular Biology of the Cell: The Problems Bo...         7559   
17    18  A Pocket Style ManualDiana HackerSt. Martin's ...         7533   
18    19  The Craft of ResearchWayne C. BoothUniversity ...         7522   
19    20    FrankensteinMary Wollstonecraft Shelley1007,225         7225   
20    21                     LeviathanThomas Hobbes1006,988         6988   
21    22                Nicomachean EthicsAristotle1006,918         6918   
22    23  A Writer's ReferenceDiana HackerSt. Martin's /...         6863   
23    24  Artificial Intelligence: A Modern ApproachS. R...         6668   
24    25  Discipline and Punish: The Birth of the Prison...         6637   
25    26               The PrinceNiccolo Machiavelli996,260         6260   
26    27  The Art of Public SpeakingStephen E. LucasMcGr...         6149   
27    28  Give Me Liberty!: An American HistoryEric Fone...         6106   
28    29              Heart of DarknessJoseph Conrad996,022         6022   
29    30          Marketing ManagementPhilip Kotler1005,995         5995   
30    31                           The RepublicPlato995,901         5901   
31    32  Research Methods for Business StudentsMark Sau...         5835   
32    33  Critical Thinking Skills: Developing Effective...         5818   
33    34  Introductory Econometrics: A Modern ApproachJe...         5803   
34    35  Easy WriterAndrea LundsfordSt. Martin's / Bedf...         5796   
35    36  Computer NetworkingJames F. Kurose, Keith W. R...         5687   
36    37  The Good Research Guide: For Small-Scale Socia...         5683   
37    38  The Clash of Civilizations?Samuel P. Huntingto...         5615   
38    39  Principles of Anatomy and PhysiologyGerard J. ...         5598   
39    40  Intermediate Algebra: A Graphing ApproachK. El...         5562   
40    41  Real World Research: A Resource for Social Sci...         5456   
41    42  Ways of Seeing: Based on the BBC Television Se...         5282   
42    43  Case Study Research: Design and MethodsRobert ...         5216   
43    44  Getting to Yes: Negotiating Agreement Without ...         5184   
44    45       Pedagogy of the OppressedPaulo Freire995,174         5174   
45    46  Intermediate AccountingDonald E. KiesoWiley985...         5145   
46    47  International Economics: Theory and PolicyPaul...         5131   
47    48  Gender Trouble: Feminism and the Subversion of...         5131   
48    49  Leadership: Theory and PracticePeter Guy North...         5098   
49    50  Doing Qualitative Research: A Practical Handbo...         5096   

    SCORE  
0     100  
1     100  
2     100  
3     100  
4     100  
5     100  
6     100  
7     100  
8     100  
9     1

## YOUR MISSION: Make playwright open a new browser and visit this page

> https://www.dmr.nd.gov/oilgas/findwellsvw.asp

In [75]:
# Imports
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [76]:
await page.goto("https://www.dmr.nd.gov/oilgas/findwellsvw.asp")

<Response url='https://www.dmr.nd.gov/oilgas/findwellsvw.asp' request=<Request url='https://www.dmr.nd.gov/oilgas/findwellsvw.asp' method='GET'>>

In [77]:
# class="barbecue", .barbecue
# id="tomato", #tomato

In [78]:
# await page.locator("select").select_option('139')
await page.get_by_label("Township:").select_option('144')

['144']

In [79]:
# find the submit button, figure out what kind of tag it is
# do await page.locator("_______").click()
# read the error message (if you get one) and fix it!!!!
# await page.locator("input").click()
await page.get_by_role("button", name="Submit").click()

In [85]:
# using 'page' you are 'clicking' how a human would actually click
import io

# Change the township
await page.get_by_label("Township:").select_option('144')
# Click the button
await page.get_by_role("button", name="Submit").click()

# Get the table
html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df.head()

0  \
0                   Related Links \tCode Definitions   
1  Data Services Notification:  January 19, 2022 ...   

                                                   1  
0  Search For Wells  Data Services Notification: ...  
1                                                NaN

## YOUR MISSION: Take my code and change it to be a different township and run it. and see how it looks. see how it works. does it work???

In [86]:
import io
import time

# Change the township
await page.get_by_label("Township:").select_option('143')
# Click the button
await page.get_by_role("button", name="Submit").click()

# WAIT A SECOND
# this is a bad way to do it :(
# Find something on the page that only shows up
# when everything is loaded and wait for it.
await page.get_by_text("CTB No").wait_for()

# Get the table
html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[2]
df.head()

File No  CTB No      API No Well Type Well Status Status Date     DTD  \
0    99016     NaN  3300399016        ST         DRY   9/20/1959     NaN   
1     9896     NaN  3300300008        OG         PNC   5/26/1983     NaN   
2     4689     NaN  3300300007        OG         DRY   6/30/1969  1370.0   
3      401     NaN  3300300003        OG         DRY   10/8/1953  1690.0   
4     9776     NaN  3309300022        OG         DRY    9/9/1982  3100.0   

         Location                         Operator  \
0       18-143-56       LEHIGH PORTLAND CEMENT CO.   
1  SWSE 14-143-57                NYVATEX OIL CORP.   
2  SWNE 29-143-58  JACK M. JOHNSTON DRILLING CORP.   
3  SESW 23-143-61                 WRIGHT & POLLARD   
4   SWSE 8-143-63          DABBLER PETROLEUM, INC.   

                      Well Name    Field  
0  GREAT NORTHERN RAILWAY CO. 9  WILDCAT  
1                TRIEBOLD 34-14  WILDCAT  
2            MATHILDA WEILAND 1  WILDCAT  
3                     GREGORY 1  WILDCAT  
4                   WANZEK 34-8  WILDCAT

In [87]:
# html = await page.content()
# tables = pd.read_html(io.StringIO(html))
# df = tables[2]
# df.head()

In [88]:
print("hello")

hello


In [89]:
for township_num in range(129, 131):
    print("Scraping", township_num)

    # Change the township
    # have to convert it to a string
    await page.get_by_label("Township:").select_option(str(township_num))
    # Click the button
    await page.get_by_role("button", name="Submit").click()

    # WAIT A SECOND
    # this is a bad way to do it :(
    # Find something on the page that only shows up
    # when everything is loaded and wait for it.
    await page.get_by_text("CTB No").wait_for()

    # Get the table
    html = await page.content()
    tables = pd.read_html(io.StringIO(html))
    df = tables[2]
    filename = f"township-{township_num}.csv"
    df.to_csv(filename, index=False)

Scraping 129
Scraping 130
